In [12]:
import  json, pprint, numpy as np
import  pandas as pd
import websocket, talib

import config
from binance.client import Client
from binance.enums import *

# unixtimestamp.com
# https://ta-lib.github.io/ta-lib-python/func_groups/momentum_indicators.html ==> RSI - Relative Strength Index 
# NOTE: The RSI function has an unstable period. real = RSI(close, timeperiod=14)
# python-binance

# Create Client
client = Client(config.APIKey, config.SecretKey)

SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.053 # 100$

closes =[]
in_position = False
my_list = []

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        #order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        order = "ok"
        print(order)
    except Exception as e:
        print(f"an exception occured - {e}")
        return False

    return True


def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('recieved message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    my_list.append(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print(f'candle closed at {close}')
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = np.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print('all rsi calculated')
            print(rsi)
            last_rsi = rsi[-1]
            print(f'the current rsi is {last_rsi}')

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    ##order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    #if order_succeeded:
                    in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    ##order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    #if order_succeeded:
                    in_position = True

ws = websocket.WebSocketApp(SOCKET, on_open=on_open , on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
recieved message
{'E': 1685574964804,
 'e': 'kline',
 'k': {'B': '0',
       'L': 1158340811,
       'Q': '626.61199300',
       'T': 1685575019999,
       'V': '0.33530000',
       'c': '1868.81000000',
       'f': 1158340795,
       'h': '1868.81000000',
       'i': '1m',
       'l': '1868.80000000',
       'n': 17,
       'o': '1868.81000000',
       'q': '1367.59119300',
       's': 'ETHUSDT',
       't': 1685574960000,
       'v': '0.73180000',
       'x': False},
 's': 'ETHUSDT'}
recieved message
{'E': 1685574967128,
 'e': 'kline',
 'k': {'B': '0',
       'L': 1158340815,
       'Q': '697.43989200',
       'T': 1685575019999,
       'V': '0.37320000',
       'c': '1868.81000000',
       'f': 1158340795,
       'h': '1868.81000000',
       'i': '1m',
       'l': '1868.80000000',
       'n': 21,
       'o': '1868.81000000',
       'q': '1466.63797200',
       's': 'ETHUSDT',
       't': 1685574960000,
       'v': '0.78480000',
       'x': False},
 's': 'ETHUSDT'}


KeyboardInterrupt: 

In [13]:
len(my_list)

494

In [14]:
import json
df = pd.json_normalize(my_list)
df.head()

,e,E,s,k.t,k.T,k.s,k.i,k.f,k.L,k.o,k.c,k.h,k.l,k.v,k.n,k.x,k.q,k.V,k.Q,k.B
0,kline,1685574964804,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340811,1868.81000000,1868.81000000,1868.81000000,1868.80000000,0.73180000,17,False,1367.59119300,0.33530000,626.61199300,0
1,kline,1685574967128,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340815,1868.81000000,1868.81000000,1868.81000000,1868.80000000,0.78480000,21,False,1466.63797200,0.37320000,697.43989200,0
2,kline,1685574970456,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340816,1868.81000000,1868.80000000,1868.81000000,1868.80000000,0.79310000,22,False,1482.14901200,0.37320000,697.43989200,0
3,kline,1685574973456,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340822,1868.81000000,1868.80000000,1868.81000000,1868.80000000,1.26220000,28,False,2358.80758400,0.82240000,1536.90934400,0
4,kline,1685574975839,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340831,1868.81000000,1868.80000000,1868.81000000,1868.80000000,3.50400000,37,False,6548.30002800,2.48280000,4639.88146800,0


In [15]:
columns = ["event", "event_time",  "symbol", "kline_open_time", "kline_close_time", "symbol", "interval", "first_trade_id", "last_trade_id",
           "open_price","close_price","highest_price","lowest_price", "base_asset_volume", "number_of_trades", "is_closed",        
           "quote_asset_volume", 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', "ignore" ]


In [16]:
df.columns = columns
df.head(2)

,event,event_time,symbol,kline_open_time,kline_close_time,symbol,interval,first_trade_id,last_trade_id,open_price,close_price,highest_price,lowest_price,base_asset_volume,number_of_trades,is_closed,quote_asset_volume,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,kline,1685574964804,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340811,1868.81000000,1868.81000000,1868.81000000,1868.80000000,0.73180000,17,False,1367.59119300,0.33530000,626.61199300,0
1,kline,1685574967128,ETHUSDT,1685574960000,1685575019999,ETHUSDT,1m,1158340795,1158340815,1868.81000000,1868.81000000,1868.81000000,1868.80000000,0.78480000,21,False,1466.63797200,0.37320000,697.43989200,0


In [18]:
df.to_csv("data/stream_data_minutely_2023_06_31.csv", index=False)

In [19]:
for i in my_list:
    

[{'e': 'kline',
  'E': 1685574964804,
  's': 'ETHUSDT',
  'k': {'t': 1685574960000,
   'T': 1685575019999,
   's': 'ETHUSDT',
   'i': '1m',
   'f': 1158340795,
   'L': 1158340811,
   'o': '1868.81000000',
   'c': '1868.81000000',
   'h': '1868.81000000',
   'l': '1868.80000000',
   'v': '0.73180000',
   'n': 17,
   'x': False,
   'q': '1367.59119300',
   'V': '0.33530000',
   'Q': '626.61199300',
   'B': '0'}},
 {'e': 'kline',
  'E': 1685574967128,
  's': 'ETHUSDT',
  'k': {'t': 1685574960000,
   'T': 1685575019999,
   's': 'ETHUSDT',
   'i': '1m',
   'f': 1158340795,
   'L': 1158340815,
   'o': '1868.81000000',
   'c': '1868.81000000',
   'h': '1868.81000000',
   'l': '1868.80000000',
   'v': '0.78480000',
   'n': 21,
   'x': False,
   'q': '1466.63797200',
   'V': '0.37320000',
   'Q': '697.43989200',
   'B': '0'}},
 {'e': 'kline',
  'E': 1685574970456,
  's': 'ETHUSDT',
  'k': {'t': 1685574960000,
   'T': 1685575019999,
   's': 'ETHUSDT',
   'i': '1m',
   'f': 1158340795,
   'L': 11

In [34]:
from pymongo import MongoClient

# Create MongoDB client
client = MongoClient('localhost', 27017)

# See all databases
client.list_database_names()

['admin', 'binance', 'config', 'local', 'sample']

In [35]:
# Create database and collection
binance = client['binance']

minutely = binance['minutely']

In [36]:
binance.list_collection_names()

['monthly', 'weekly', 'minutely', 'hourly', 'daily']

In [29]:
for i in list(client['binance']['monthly'].aggregate([{"$group": {"_id" : "$id_asset", "nb" : {"$sum" : 1}}}])):
    print(i)

{'_id': 2, 'nb': 71}
{'_id': 1, 'nb': 70}
{'_id': 3, 'nb': 70}


In [33]:
for i in my_list:
        client['binance']['minutely'].insert_one(i)

client['binance']['minutely'].find_one()

{'_id': ObjectId('6477e0884a9f7746b16e3e8c'),
 'e': 'kline',
 'E': 1685574964804,
 's': 'ETHUSDT',
 'k': {'t': 1685574960000,
  'T': 1685575019999,
  's': 'ETHUSDT',
  'i': '1m',
  'f': 1158340795,
  'L': 1158340811,
  'o': '1868.81000000',
  'c': '1868.81000000',
  'h': '1868.81000000',
  'l': '1868.80000000',
  'v': '0.73180000',
  'n': 17,
  'x': False,
  'q': '1367.59119300',
  'V': '0.33530000',
  'Q': '626.61199300',
  'B': '0'}}